## Process the data

#### 1.Split the data

In [1]:
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

In [2]:
Path = './'
df = pd.read_csv(Path + 'train_targets_scored.csv')

In [3]:
df.head(2)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.loc[:,'kfold'] = -1

In [5]:
df = df.sample(frac=1).reset_index(drop=True)
targets = df.drop(['sig_id'],axis=1).values

In [6]:
mskf = MultilabelStratifiedKFold(n_splits=5,shuffle=False, random_state=None)

/Users/bat/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=False, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [7]:
df.head(2)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_ae71951bd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
1,id_e88e993ff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1


In [8]:
for fold, (trn_, val_) in enumerate(mskf.split(X=df,y=targets)):
    df.loc[val_,"kfold"] = fold

In [9]:
df.head(2)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_ae71951bd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,id_e88e993ff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
df['kfold'].value_counts()

4    4763
3    4763
2    4763
0    4763
1    4762
Name: kfold, dtype: int64

In [11]:
df.to_csv('train_fold.csv',index = False)

In [12]:
def add_dummies(data, column):
    ohe = pd.get_dummies(data[column])
    ohe_columns = [f"{column}_{c}" for c in ohe.columns]
    ohe.columns = ohe_columns
    data = data.drop(column, axis=1)
    data = data.join(ohe)
    return data

In [13]:
def process_data(df):
    df = add_dummies(df, 'cp_time')
    df = add_dummies(df, 'cp_type')
    df = add_dummies(df, 'cp_dose')
    
    return df

## Modeling

In [14]:
import torch
import torch.nn as nn

In [15]:
class MoaDataset():
    def __init__(self, dataset, features):
        self.dataset = dataset
        self.features = features
    def __len__(self):
        return self.dataset.shape[0]
    
    def __getitem__(self,item):
        return {
            "x" : torch.tensor(self.dataset[item,:],dtype=torch.float),
            "y" : torch.tensor(self.features[item,:],dtype=torch.float)
        }

In [16]:
#from tqdm.notebook import tqdm
from tqdm import tqdm 

In [35]:
class Engine():
    def __init__(self, model, optimizer, device):
        self.model = model
        self.optimizer = optimizer
        self.device = device
    
    def loss_fn(self, targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs,targets)

    def train(self, data_loader):
        self.model.train()
        final_loss = 0 
        for data in tqdm(data_loader):
            self.optimizer.zero_grad()
            inputs = data['x'].to(self.device)
            targets = data['y'].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            
            loss.backward()
            self.optimizer.step()
            
            final_loss += loss.item()
        print('Training loss : {}'.format(final_loss))
        return final_loss / len(data_loader)
    
    def validation(self, data_loader):
        self.model.eval()
        final_loss = 0 
        for data in tqdm(data_loader):
            inputs = data['x'].to(self.device)
            targets = data['y'].to(self.device)
            
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            final_loss += loss.item()

        print('validation loss : {}'.format(final_loss))
        return final_loss / len(data_loader)
            

In [36]:
EPOCHS = 100
DEVICE = 'cpu'

In [37]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.3),
            nn.PReLU(),
            
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.3),
            nn.PReLU(),
            
            nn.Linear(1024, num_targets)
            )
    def forward(self, x):
        x = self.model(x)
        return x
        

In [38]:
def run_training(fold):
    df = pd.read_csv(Path + 'train_features.csv')
    df = process_data(df)
    folds = pd.read_csv(Path + 'train_fold.csv')
    
    targets = folds.drop(['sig_id','kfold'], axis =1 ).columns
    features = df.drop(['sig_id'],axis=1).columns
    
    df = df.merge(folds, on = 'sig_id', how='left')
    
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    
    X_train = train_df[features].values
    y_train = train_df[targets].values
    
    X_valid = valid_df[features].values
    y_valid = valid_df[targets].values
    
    
    train_dataset = MoaDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(
                    train_dataset, batch_size=64, num_workers = 8)
    
    valid_dataset = MoaDataset(X_valid, y_valid)
    valid_loader = torch.utils.data.DataLoader(
                    valid_dataset, batch_size=64, num_workers = 8)
    
    model = Model(X_train.shape[1],y_train.shape[1])
    model = model.to(DEVICE)
    
    optmizer = torch.optim.Adam(model.parameters(), lr = 3e-4)
    # scheduler = 
    eng = Engine(model, optmizer, DEVICE)
    
    for epoch in (range(EPOCHS)):
        print('epoch : {}'.format(epoch))
        train_loss = eng.train(train_loader)
        valid_loss = eng.validation(valid_loader)
    
    print('for fold : {} train loss = {}, validation loss : {} '.format(fold, train_loss, valid_loss))

In [39]:
run_training(1)

  0%|          | 0/298 [00:00<?, ?it/s]

epoch : 0


  0%|          | 0/75 [00:00<?, ?it/s]

Training loss : 17.842223745770752


  0%|          | 0/298 [00:00<?, ?it/s]

...

Training loss : 0.3086437392339576


  0%|          | 0/298 [00:00<?, ?it/s]

validation loss : 1.9099840074777603
epoch : 32


  2%|▏         | 5/298 [00:01<00:58,  4.99it/s]


KeyboardInterrupt: 